In [1]:
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('../data/US_Accidents_March23_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127862 entries, 0 to 127861
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Severity               127862 non-null  int64  
 1   Start_Lat              127862 non-null  float64
 2   Start_Lng              127862 non-null  float64
 3   Street                 127862 non-null  object 
 4   City                   127862 non-null  object 
 5   County                 127862 non-null  object 
 6   State                  127862 non-null  object 
 7   Zipcode                127862 non-null  object 
 8   Timezone               127862 non-null  object 
 9   Airport_Code           127862 non-null  object 
 10  Temperature(F)         127862 non-null  float64
 11  Humidity(%)            127862 non-null  float64
 12  Pressure(in)           127862 non-null  float64
 13  Visibility(mi)         127862 non-null  float64
 14  Wind_Direction         127862 non-nu

In [3]:
df = df.drop(['Zipcode', 'Airport_Code','Street'], axis=1)

In [4]:
df.columns

Index(['Severity', 'Start_Lat', 'Start_Lng', 'City', 'County', 'State',
       'Timezone', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset', 'Civil_Twilight',
       'Nautical_Twilight', 'Astronomical_Twilight', 'Clear', 'Cloud', 'Rain',
       'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'Precipitation_NA'],
      dtype='object')

In [5]:
df_dummy = pd.get_dummies(df, drop_first=True)

In [6]:
target='Severity'

y = df_dummy[target]
X = df_dummy.drop(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

# write model below

In [7]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

acc=accuracy_score(y_test, y_pred)
print("[Logistic regression algorithm] accuracy_score: {:.3f}.".format(acc))

e:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[Logistic regression algorithm] accuracy_score: 0.795.


In [8]:
# joblib.dump(lr, '../models/lr_model.pkl')

In [9]:
knn = KNeighborsClassifier(n_neighbors=6)
X_train_smaller, _, y_train_smaller, _ = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
knn.fit(X_train_smaller, y_train_smaller)
X_test_Knn = X_test.reset_index(drop=True)
X_test_Knn = X_test_Knn.values
y_pred_knn = knn.predict(X_test_Knn)

e:\anaconda\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [10]:
acc_knn = accuracy_score(y_test, y_pred_knn)
print("[KNN algorithm] accuracy_score: {:.3f}".format(acc_knn))

[KNN algorithm] accuracy_score: 0.786


In [ ]:
joblib.dump(knn, '../models/knn_model.pkl')